In [1]:
# Decorator : 특정한 function, method를 꾸며주는 것
# function의 앞, 뒤로 해야 할 일이나 로깅, 벤치마킹 등 다양한 용도로 쓰일 수 있음
# 어떻게 가능한지?
# First-class function이기 때문(파이썬이 함수를 일급시민으로 취급하는 것)
# 함수를 다른 함수의 인자로 전달하거나, 결과값으로 return 가능
# 함수를 변수에 할당하거나 데이터 영역에 저장 가능

In [2]:
 def make_difference(operator):
        if operator == '+':
            return lambda x,y:x+y
        elif operator == '-':
            return lambda x,y:x-y

In [9]:
plus = make_difference('+')   # 함수를 변수에 할당 가능, 람다는 원래 heap에 저장되는데 전역 변수에 할당이 되어 데이터 영역에 저장
plus

<function __main__.make_difference.<locals>.<lambda>(x, y)>

In [6]:
type(plus)

function

In [8]:
plus(1,2)

3

In [11]:
# 함수를 변수에 할당하면 그 변수도 함수가 되어 일급시민으로 취급됨
def print_hello(msg):
    print(msg)
    
copied = print_hello
print(copied, print_hello)    # call by object reference, 할당되어있는 주소값이 같음

<function print_hello at 0x7f2e20551550> <function print_hello at 0x7f2e20551550>


In [10]:
# Closure: First-class function을 지원하는 언어의 네임을 바인딩(변수에 값을 저장)할 때 사용
# : 함수와 함께 함수 자신의 주변 환경을 저장한 레코드를 클로저라고 부름
def outer():
    text_a = 'John'
    def inner():
        b = 'Doe'
        print("My name is {} {}.".format(text_a,b))
    return inner    # 함수 객체만 전달, inner()는 함수를 수행하는 의미로 값이 날라감

In [11]:
outer()  # inner()를 호출했기 때문에

<function __main__.outer.<locals>.inner()>

In [15]:
func = outer()   # inner객체가 전달되서 func에 inner()이 있음
func             # First-class function으로 func도 함수가 됨

<function __main__.outer.<locals>.inner()>

In [16]:
func.__dir__()  # inner가 수행되기 위한 메소드들을 확인하고, 레코드를 조회하기 위해 closure에 접근해야 함

['__repr__',
 '__call__',
 '__get__',
 '__new__',
 '__closure__',
 '__doc__',
 '__globals__',
 '__module__',
 '__code__',
 '__defaults__',
 '__kwdefaults__',
 '__annotations__',
 '__dict__',
 '__name__',
 '__qualname__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [19]:
func.__closure__  # 튜플 형태이고, 0번째 값이 있음

(<cell at 0x7f35b85281c0: str object at 0x7f35b851ebf0>,)

In [24]:
func.__closure__[0].cell_contents # 값을 조회해 보니, 'John'이 저장되어 있음. 
                                  # 왜 값을 저장해 두었는지? inner에서 text_a를 참조하니까 inner가 실행될 때 기억하고 있어야 함
                                  # closure라는 기능이 없었다면 함수가 실행되지 못했을 것
                                  # closure가 함수와 주변부를 레코드로 저장해 두었기 때문에 inner가 수행할 떄 그 레코드에서 값을 가져다 사용함

'John'

In [12]:
# decorator: 어떤 함수 위에 적어줌, 구조는 inner function을 만들고 inner function을 리턴해야 함
# inner == 객체 소환, inner() == inner 수행
def rapper(func):                  # 함수 객체 만들고
    def wrapper():                 # inner function 만들고
        print('너와나의 연결고리..')   #                        ----- 순서 1.
        func()                     # 메인function 수행        ----- 순서 2.
    return wrapper                 # inner function 리턴하고

@rapper #실제로는 wraapper이 실행됨    # 데코레이터의 역할: rapper()을 기준으로 앞에 할 일, 뒤에 할 일을 정의하고 싶을 때 사용
def dok2():                        # main function
    print('이건 우리 안의 소리')

In [13]:
dok2()

너와나의 연결고리..
이건 우리 안의 소리


In [170]:
def rapper(func):                  # 함수 객체 만들고
    def wrapper():                 # inner function 만들고
        print('너와나의 연결고리..')
        func()                     # 메인function 수행
    return wrapper                 # inner function 리턴하고

#@rapper                           # 데코레이터의 역할: rapper()을 기준으로 앞에 할 일, 뒤에 할 일을 정의하고 싶을 때 사용
def dok2():                        # main function
    print('이건 우리 안의 소리')

In [171]:
rapper(dok2)()  # 데코레이터 없이 호출하는 방법

너와나의 연결고리..
이건 우리 안의 소리


In [110]:
def fibo_rec(num):
    if num < 2:
        return num
    else:
        return fibo_rec(num-2) + fibo_rec(num-1)

In [111]:
fibo_rec(3)   # 계산 속도 오래걸림

2

In [116]:
# 피보나치를 재귀함수로 구현하면 조회해야 하는 값이 반복되서 시간이 오래 걸리기 떄문에 데코레이터를 활용하여 값을 한번만 조회하게 구현
def memoize(func):
    memo = {}     # closure라 레코드가 저장됨
    def wrapper(seq):
        print('seq는 {}'.format(seq))
        if seq not in memo:            
            memo[seq] = func(seq)
            print('{}: '.format(seq))
            print(memo)
        return memo[seq]     
    return wrapper

@memoize
def fib_memo(num):
    if num<2:
        return num
    else:
        return fib_memo(num-1) + fib_memo(num-2)

In [117]:
fib_memo(3)

seq는 3
seq는 2
seq는 1
1: 
{1: 1}
seq는 0
0: 
{1: 1, 0: 0}
2: 
{1: 1, 0: 0, 2: 1}
seq는 1
3: 
{1: 1, 0: 0, 2: 1, 3: 2}


2

In [ ]:
# practice : 1호차 입니다. ~ 5호차 입니다. 반복문 X

In [145]:
# 반복되는 코드가 많아 데코레이터 사용 목적이 없어짐
def train_1(func):
    def wrapper():
        print('1호차 입니다.')
        func()
    return wrapper

def train_2(func):
    def wrapper():
        print('2호차 입니다.')
        func()
    return wrapper

def train_3(func):
    def wrapper():
        print('3호차 입니다.')
        func()
    return wrapper

def train_4(func):
    def wrapper():
        print('4호차 입니다.')
        func()
    return wrapper


@train_1
@train_2
@train_3
@train_4
def print_train():
    print('5호차 입니다.')

In [146]:
print_train()

1호차 입니다.
2호차 입니다.
3호차 입니다.
4호차 입니다.
5호차 입니다.


In [154]:
from  time import time 

def time_checker(func):
    def wrapper(seq):
        start_at = time()
        result = func(seq)
        end_at = time()
        print('연산 시간: {}sec'.format(end_at-start_at))
        return result
    return wrapper
              
              
@time_checker
def fibo_rec(num):
    if num < 2:
        return num
    else:
        return fibo_rec(num-1) + fibo_rec(num-2)
              

In [155]:
fibo_rec(10)

연산 시간: 4.76837158203125e-07sec
연산 시간: 4.76837158203125e-07sec
연산 시간: 0.00010848045349121094sec
연산 시간: 7.152557373046875e-07sec
연산 시간: 0.0002684593200683594sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 4.76837158203125e-07sec
연산 시간: 4.363059997558594e-05sec
연산 시간: 0.00035381317138671875sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 4.76837158203125e-07sec
연산 시간: 4.1484832763671875e-05sec
연산 시간: 9.5367431640625e-07sec
연산 시간: 0.0005893707275390625sec
연산 시간: 0.0009946823120117188sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 4.76837158203125e-07sec
연산 시간: 4.482269287109375e-05sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 8.559226989746094e-05sec
연산 시간: 4.76837158203125e-07sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 4.076957702636719e-05sec
연산 시간: 0.00016736984252929688sec
연산 시간: 0.0012030601501464844sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 4.00543212890625e-05sec
연산 시간: 4.76837158203125e-07sec
연산 시간: 0.0003998279571533203sec
연산 시간: 2.384185791015625e-07sec
연산 시간: 4.76837

55

In [159]:
from  time import time 

def time_checker(func):
    def wrapper(seq):
        start_at = time()
        result = func(seq)
        end_at = time()
        print('연산 시간: {}sec'.format(end_at-start_at))
        return result
    return wrapper
              
              
#@time_checker
def fibo_rec(num):
    if num < 2:
        return num
    else:
        return fibo_rec(num-1) + fibo_rec(num-2)

In [160]:
time_checker(fibo_rec)(10)

연산 시간: 6.413459777832031e-05sec


55